# Back-Test of 3M vol Swap (Start from 2010), group by Sectors
Vol Swap 3M, payoff = RV(3M) - IV(3M)

Rebalance every week (5 trading days).

Trading universe is SP500 on a rolling base, and is grouped into 3 sub-universe: small(<10b), mid(10b~50b), large(>50b)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

# Load Data

In [2]:
directory = "h:/test_2005/"

trading_dates = pd.read_csv(directory + "trading_dates.csv", index_col=0)
trading_dates["MarketDate"] = pd.to_datetime(trading_dates["MarketDate"])

data = pd.read_csv(directory+"rolling_universe.csv", index_col=0)["Infocode"]
data.index = pd.to_datetime(data.index)
rolling_universe = {}
for key in data.index:
    rolling_universe[key] = np.array(data[key].split()).astype('int')
    
infocode_dict = pd.read_csv(directory + "infocode_dict.csv", index_col=0)["Ticker"].to_dict()
infocode_to_trid = pd.read_csv(directory + "infocode_to_trid.csv", index_col=0)["ID"].to_dict()
ticker_dict = pd.read_csv(directory + "ticker_dict.csv", index_col=0)["Infocode"].to_dict()
infocode_sector = pd.read_csv(directory + "infocode_sector.csv", index_col=0)["ECONSECT"].to_dict()

In [3]:
def read_stock_data(directory, name):
    stock_data = pd.read_csv(directory + name, index_col=0)
    stock_data.index = pd.to_datetime(stock_data.index)
    stock_data.columns = stock_data.columns.astype("int")
    return stock_data

In [4]:
stock_prices = read_stock_data(directory, 'stock_prices.csv')
stock_iv3m = read_stock_data(directory, 'stock_iv3m.csv')
stock_iv6m = read_stock_data(directory, 'stock_iv6m.csv')
stock_iv12m = read_stock_data(directory, 'stock_iv12m.csv')
stock_iv2m = read_stock_data(directory, 'stock_iv2m.csv')
stock_iv1m = read_stock_data(directory, 'stock_iv1m.csv')
stock_iv1m50delta = read_stock_data(directory, 'stock_iv1m50delta.csv')
stock_iv1m25delta = read_stock_data(directory, 'stock_iv1m25delta.csv')
stock_iv1m75delta = read_stock_data(directory, 'stock_iv1m75delta.csv')
stock_iv2m50delta = read_stock_data(directory, 'stock_iv2m50delta.csv')
stock_iv2m25delta = read_stock_data(directory, 'stock_iv2m25delta.csv')
stock_iv2m75delta = read_stock_data(directory, 'stock_iv2m75delta.csv')

In [5]:
stock_asset_per_equity = read_stock_data(directory, 'stock_asset_per_equity.csv')
stock_asset_per_marketcap = read_stock_data(directory, 'stock_asset_per_marketcap.csv')
stock_eps = read_stock_data(directory, 'stock_eps.csv')
stock_roe = read_stock_data(directory, 'stock_roe.csv')
stock_cps = read_stock_data(directory, 'stock_cps.csv')
stock_cpx_per_marketcap = read_stock_data(directory, 'stock_cpx_per_marketcap.csv')
stock_dps = read_stock_data(directory, 'stock_dps.csv')
stock_marketcap = read_stock_data(directory, 'stock_marketcap.csv')

In [6]:
trading_interval = 5
holding_period = 63

start_date = pd.to_datetime('20100101')
end_date = trading_dates["MarketDate"].iloc[-1]

rebalance_dates = []
valid_dates = trading_dates[(trading_dates["MarketDate"] >= start_date) & (trading_dates["MarketDate"] <= end_date)]
for i in range(0, len(valid_dates), trading_interval):
    rebalance_dates.append(valid_dates["MarketDate"].iloc[i])
    
trading_dates = list(pd.to_datetime(trading_dates["MarketDate"].values))

In [7]:
stock_returns = np.log(stock_prices/stock_prices.shift())

In [8]:
stock_rv1m = pd.rolling_std(stock_returns, 21) * np.sqrt(252)
stock_rv2m = pd.rolling_std(stock_returns, 42) * np.sqrt(252)
stock_rv3m = pd.rolling_std(stock_returns, 63) * np.sqrt(252)
stock_rv6m = pd.rolling_std(stock_returns, 126) * np.sqrt(252)
stock_rv12m = pd.rolling_std(stock_returns, 252) * np.sqrt(252)
stock_rv10d = pd.rolling_std(stock_returns, 10) * np.sqrt(252)

In [ ]:
vix = pd.read_csv(directory + 'vix.csv', index_col=0)
vix = pd.Series(vix['vix'].values, index=pd.to_datetime(vix.index))
vix_ma = pd.rolling_mean(vix, 10)

In [ ]:
rolling_universe_small = {}
rolling_universe_mid = {}
rolling_universe_large = {}
for date in trading_dates:
    MC = stock_marketcap.loc[date][rolling_universe[date]]
    rolling_universe_small[date] = np.array(MC[MC<10000].index)
    rolling_universe_mid[date] = np.array(MC[(MC<50000) & (MC>10000)].index)
    rolling_universe_large[date] = np.array(MC[MC>50000].index)

In [ ]:
infocode_sector[72990]

In [ ]:
infocode_sectors = pd.Series(infocode_sector)
sectors = ['Financials', 'Consumer Cyclicals', 'Industrials', 'Technology', 'Healthcare', 'Consumer Non-Cyclicals', 'Energy']
rolling_universe_sectors = {}
for sector in sectors:
    rolling_universe_sectors[sector] = {}
for date in trading_dates:
    universe = infocode_sectors[rolling_universe[date]]
    for sector in sectors:
        if sector != 'Others':
            rolling_universe_sectors[sector][date] = np.array(universe[universe == sector].index)
        else:
            rolling_universe_sectors[sector][date] = np.array(universe[(universe == 'Utilities') | (universe == 'Basic Materials') | (universe == 'Telecommunications Services') | (universe == 'Others')].index)

# Back Test Result

In [ ]:
def backtest3m(score):
    daily_pnl = pd.Series(np.zeros(len(trading_dates)), index=trading_dates)
    for i in range(len(trading_dates)):
        date = trading_dates[i]
        #print date
        if (date in rebalance_dates) & (i + holding_period < len(trading_dates)):
            z = score.loc[date][rolling_universe[date]].dropna()
            z = (z-z.mean()) / z.std()
            z.sort(ascending=False)
            long_short = np.append(z.index[ : 1*len(z)/10], z.index[-(1*len(z)/10) : ])
            for infocode in long_short:
                if not np.isnan(stock_iv3m[infocode][date]):
                    daily_pnl[trading_dates[i+holding_period]] += (100 / len(long_short)) * np.sign(z[infocode]) * (stock_rv3m[infocode][trading_dates[i+holding_period]] - stock_iv3m[infocode][date])
    return daily_pnl

def backtest3m_sectors(score, sector):
    daily_pnl = pd.Series(np.zeros(len(trading_dates)), index=trading_dates)
    for i in range(len(trading_dates)):
        date = trading_dates[i]
        #print date
        if (date in rebalance_dates) & (i + holding_period < len(trading_dates)):
            z = score.loc[date][rolling_universe_sectors[sector][date]].dropna()
            z = (z-z.mean()) / z.std()
            z.sort(ascending=False)
            long_short = np.append(z.index[ : 1*len(z)/10], z.index[-(1*len(z)/10) : ])
            for infocode in long_short:
                if not np.isnan(stock_iv3m[infocode][date]):
                    daily_pnl[trading_dates[i+holding_period]] += (100 / len(long_short)) * np.sign(z[infocode]) * (stock_rv3m[infocode][trading_dates[i+holding_period]] - stock_iv3m[infocode][date])
    return daily_pnl


In [ ]:
def sharpe_ratio(pnl):
    return pnl[pnl != 0].mean() / pnl[pnl != 0].std() * np.sqrt(12/3)

def pnl_mean(pnl):
    return pnl[pnl != 0].mean()

def pnl_std(pnl):
    return pnl[pnl != 0].std()

def hit_rate(pnl):
    return float(len(pnl[pnl > 0])) / len(pnl[pnl != 0])

def pnl_min(pnl):
    return pnl[pnl != 0].min()

def pnl_25perc(pnl):
    return pnl[pnl != 0].quantile(0.25)

def pnl_median(pnl):
    return pnl[pnl != 0].median()

def pnl_75perc(pnl):
    return pnl[pnl != 0].quantile(0.75)

def pnl_max(pnl):
    return pnl[pnl != 0].max()

def pnl_days(pnl):
    return len(pnl[pnl != 0])

In [ ]:
def report_table(pnls, sectors):
    result = pd.DataFrame(index = sectors, columns=['Sharpe Ratio', 'PnL Mean', 'PnL Std', 'Hit Rate', 'PnL Min', 'PnL 25%Perc', 'PnL Median', 'PnL 75%Perc', 'PnL Max'])
    for sector in sectors:
        result.loc[sector]['Sharpe Ratio'] = sharpe_ratio(pnls[sector])
        result.loc[sector]['PnL Mean'] = pnl_mean(pnls[sector])
        result.loc[sector]['PnL Std'] = pnl_std(pnls[sector])
        result.loc[sector]['Hit Rate'] = hit_rate(pnls[sector])
        result.loc[sector]['PnL Min'] = pnl_min(pnls[sector])
        result.loc[sector]['PnL 25%Perc'] = pnl_25perc(pnls[sector])
        result.loc[sector]['PnL Median'] = pnl_median(pnls[sector])
        result.loc[sector]['PnL 75%Perc'] = pnl_75perc(pnls[sector])
        result.loc[sector]['PnL Max'] = pnl_max(pnls[sector])
    return result

In [ ]:
def annual_pnl_mean(pnls, sectors):
    years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
    result = pd.DataFrame(index = sectors, columns=years)
    for sector in sectors:
        for year in years:
            result.loc[sector][year] = pnl_mean(pnls[sector][pd.to_datetime(year+'0101') : pd.to_datetime(year+'1231')])
    result.fillna(0, inplace=True)
    return result

In [ ]:
def annual_trade_days(pnls, labels):
    years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
    result = pd.DataFrame(index = sectors, columns=years)
    for i in range(len(pnls)):
        for year in years:
            result.loc[labels[i]][year] = pnl_days(pnls[i][pd.to_datetime(year+'0101') : pd.to_datetime(year+'1231')])
    result.fillna(0, inplace=True)
    return result

In [ ]:
def annual_stock_numbers(labels=['small(<10B)','mid(10B~50B)','large(>50B)']):
    years = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
    result = pd.DataFrame(index = labels, columns=years)
    for year in years:
        date = np.array(trading_dates)[np.array(trading_dates) > pd.to_datetime(year+'0101')][0]
        total = float(len(rolling_universe[date]))
        result.loc['small(<10B)'][year] = len(rolling_universe_small[date]) / total
        result.loc['mid(10B~50B)'][year] = len(rolling_universe_mid[date]) / total
        result.loc['large(>50B)'][year] = len(rolling_universe_large[date]) / total
    return result

## RV-IV

In [ ]:
pnls = {}
for sector in sectors:
    pnls[sector] = backtest3m_sectors(stock_rv3m - stock_iv3m, sector)

In [ ]:
plt.figure(figsize=(8, 4.5))
for sector in sectors:
    plt.plot(pnls[sector].cumsum(), label=sector)
plt.xlim(pd.to_datetime('20100101'), pd.to_datetime('20180601'))
plt.title("Cumulative PnL")
plt.grid(True)
plt.legend(loc='upper left', frameon=True, fontsize='small')
plt.ylim(-500, 1000)
plt.savefig('h:/rv-iv.png')
plt.show()

In [ ]:
performance = report_table(pnls, sectors)
performance

In [ ]:
pnl_mean_yearly = annual_pnl_mean(pnls, sectors)
pnl_mean_yearly

## Vol Skew

In [ ]:
pnls = {}
for sector in sectors:
    pnls[sector] = backtest3m_sectors((stock_iv2m75delta - stock_iv2m25delta) / stock_iv1m50delta, sector)

In [ ]:
plt.figure(figsize=(8, 4.5))
for sector in sectors:
    plt.plot(pnls[sector].cumsum(), label=sector)
plt.xlim(pd.to_datetime('20100101'), pd.to_datetime('20180601'))
plt.title("Cumulative PnL")
plt.grid(True)
plt.legend(loc='upper left', frameon=True, fontsize='small')
plt.ylim(-500, 1000)
plt.savefig('h:/rv-iv.png')
plt.show()

In [ ]:
performance = report_table(pnls, sectors)
performance

In [ ]:
pnl_mean_yearly = annual_pnl_mean(pnls, sectors)
pnl_mean_yearly

## Term Structure

In [ ]:
pnls = {}
for sector in sectors:
    pnls[sector] = backtest3m_sectors(- stock_iv3m + stock_iv6m, sector)

In [ ]:
plt.figure(figsize=(8, 4.5))
for sector in sectors:
    plt.plot(pnls[sector].cumsum(), label=sector)
plt.xlim(pd.to_datetime('20100101'), pd.to_datetime('20180601'))
plt.title("Cumulative PnL")
plt.grid(True)
plt.legend(loc='upper left', frameon=True, fontsize='small')
plt.ylim(-500, 1000)
plt.savefig('h:/rv-iv.png')
plt.show()

In [ ]:
performance = report_table(pnls, sectors)
performance

In [ ]:
pnl_mean_yearly = annual_pnl_mean(pnls, sectors)
pnl_mean_yearly

## Cash Bollinger

In [ ]:
pnls = {}
for sector in sectors:
    pnls[sector] = backtest3m_sectors(np.abs((stock_prices - pd.rolling_mean(stock_prices, 378)) / pd.rolling_std(stock_prices, 378)), sector)

In [ ]:
plt.figure(figsize=(8, 4.5))
for sector in sectors:
    plt.plot(pnls[sector].cumsum(), label=sector)
plt.xlim(pd.to_datetime('20100101'), pd.to_datetime('20180601'))
plt.title("Cumulative PnL")
plt.grid(True)
plt.legend(loc='upper left', frameon=True, fontsize='small')
plt.ylim(-500, 1000)
plt.savefig('h:/rv-iv.png')
plt.show()

In [ ]:
performance = report_table(pnls, sectors)
performance

In [ ]:
pnl_mean_yearly = annual_pnl_mean(pnls, sectors)
pnl_mean_yearly

## Cash RSI

In [ ]:
stock_delta1 = stock_prices - stock_prices.shift(1)
stock_gain = stock_delta1[stock_delta1>0]
stock_loss = np.abs(stock_delta1[stock_delta1<0])

def stock_rsi(N):
    return 100 * 1 / (1 + pd.rolling_mean(stock_loss, N, min_periods=0) / pd.rolling_mean(stock_gain, N, min_periods=0))

In [ ]:
pnls = {}
for sector in sectors:
    pnls[sector] = backtest3m_sectors(-stock_rsi(63), sector)

In [ ]:
plt.figure(figsize=(8, 4.5))
for sector in sectors:
    plt.plot(pnls[sector].cumsum(), label=sector)
plt.xlim(pd.to_datetime('20100101'), pd.to_datetime('20180601'))
plt.title("Cumulative PnL")
plt.grid(True)
plt.legend(loc='upper left', frameon=True, fontsize='small')
plt.ylim(-500, 1000)
plt.savefig('h:/rv-iv.png')
plt.show()

In [ ]:
performance = report_table(pnls, sectors)
performance

In [ ]:
pnl_mean_yearly = annual_pnl_mean(pnls, sectors)
pnl_mean_yearly

## CPX/MC

In [ ]:
pnls = {}
for sector in sectors:
    pnls[sector] = backtest3m_sectors(stock_cpx_per_marketcap, sector)

In [ ]:
plt.figure(figsize=(8, 4.5))
for sector in sectors:
    plt.plot(pnls[sector].cumsum(), label=sector)
plt.xlim(pd.to_datetime('20100101'), pd.to_datetime('20180601'))
plt.title("Cumulative PnL")
plt.grid(True)
plt.legend(loc='upper left', frameon=True, fontsize='small')
plt.ylim(-500, 1000)
plt.savefig('h:/rv-iv.png')
plt.show()

In [ ]:
performance = report_table(pnls, sectors)
performance

In [ ]:
pnl_mean_yearly = annual_pnl_mean(pnls, sectors)
pnl_mean_yearly

## DPS

In [ ]:
pnls = {}
for sector in sectors:
    pnls[sector] = backtest3m_sectors(stock_dps, sector)

In [ ]:
plt.figure(figsize=(8, 4.5))
for sector in sectors:
    plt.plot(pnls[sector].cumsum(), label=sector)
plt.xlim(pd.to_datetime('20100101'), pd.to_datetime('20180601'))
plt.title("Cumulative PnL")
plt.grid(True)
plt.legend(loc='upper left', frameon=True, fontsize='small')
plt.ylim(-500, 1000)
plt.savefig('h:/rv-iv.png')
plt.show()

In [ ]:
performance = report_table(pnls, sectors)
performance

In [ ]:
pnl_mean_yearly = annual_pnl_mean(pnls, sectors)
pnl_mean_yearly